# Best Neighborhoods in South King County for Seattle Workers
### IBM Data Science Capstone Project - Battle of the Neighborhoods
Elizabeth Roenicke - December 2020

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

The city of Seattle in Washington State is an expensive place to live! The housing market continues to flourish, attracting men and women who are looking to live in the big city. 

However, there are many opportunities to live in South King County with the amenities a Seattlite might find attractive, while paying the lower costs associated with being outside the city proper.

This project seeks to **determine which areas in South King County provide the best amenities for commuters with jobs in Seattle**. We will look at areas in South King County by city and zip code and will assess their abundance of 4 main amenities, namely medical facilities, police stations, Metro park-and-rides, and restaurants. 

We will **give each area a score** based on these amenities. **Homebuyers can use these scores** as a springboard for their home searches, and **realtors can use these scores** as selling points to Seattle commuters for their properties: providing a taste of the big city for a fraction of the price.

## Data <a name='data'></a>

We will begin with a masterlist of zip code data that includes fields we will need later for analysis. We will be using a csv files downloaded from the King County GIS Open Data website. Here is a link to the [zip code file](https://gis-kingcounty.opendata.arcgis.com/datasets/addresses-in-king-county-address-point/data).

In [1]:
# Start by importing all the necessary for this notebook

import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

Libraries imported.


In [2]:
# Read the zip code csv file with pandas

df_zip = pd.read_csv('Zip_Codes.csv')
df_zip.head()

C:\Users\Kalaha\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6,13,23,25,27,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,X,Y,OBJECTID,COMMENTS,SITETYPE,Alias1,Alias2,Alias3,Alias4,Alias5,SITEID,ADDR_HN,ADDR_PD,ADDR_PT,ADDR_SN,ADDR_ST,ADDR_SD,ADDR_NUM,ADDR_FULL,FULLNAME,COMPRESS_NAME,COMPRESS_ADDR,ZIPSECTORL,ZIPSEG_L,ZIPSECTORH,ZIPSEG_H,ZIP5,PLUS4,CR_ID,UPDATE_KEY,CTYNAME,POSTALCTYNAME,LAT,LON,PRIM_ADDR,PRIM_ADDR_FILTER,SITETYPE_DESCRIPTION,POINT_X,POINT_Y,COUNTY,MAJOR,MINOR,PIN
0,-122.358053,47.656113,1,NaN,R1,NaN,NaN,NaN,NaN,NaN,324731,119,NW,NaN,41ST,ST,NaN,119,119 NW 41ST ST,NW 41ST ST,NW41STST,119NW41STST,49.0,28,49.0,28,98107,4928,C010,Z107775400,SEATTLE,SEATTLE,47.656113,-122.358053,1,ESITE:ONETOONE,Single Family,1.264675e+06,243052.327669,KING,990600.0,0100,9906000100
1,-122.357894,47.656114,2,NaN,R1,NaN,NaN,NaN,NaN,NaN,316864,115,NW,NaN,41ST,ST,NaN,115,115 NW 41ST ST,NW 41ST ST,NW41STST,115NW41STST,49.0,28,49.0,28,98107,4928,C010,Z107775400,SEATTLE,SEATTLE,47.656114,-122.357894,1,ESITE:ONETOONE,Single Family,1.264714e+06,243051.663628,KING,990600.0,0090,9906000090
2,-122.357731,47.656114,3,NaN,R1,NaN,NaN,NaN,NaN,NaN,331314,111,NW,NaN,41ST,ST,NaN,111,111 NW 41ST ST,NW 41ST ST,NW41STST,111NW41STST,49.0,28,49.0,28,98107,4928,C010,Z107775400,SEATTLE,SEATTLE,47.656114,-122.357731,1,ESITE:ONETOONE,Single Family,1.264754e+06,243050.987777,KING,990600.0,0080,9906000080
3,-122.357475,47.656015,4,NaN,R1,NaN,NaN,NaN,NaN,NaN,323422,4015,NaN,NaN,1ST,AVE,NW,4015,4015 1ST AVE NW,1ST AVE NW,1STAVENW,40151STAVENW,49.0,8,49.0,8,98107,4908,C010,Z107774670,SEATTLE,SEATTLE,47.656015,-122.357475,1,ESITE:ONETOONE,Single Family,1.264816e+06,243013.791001,KING,990600.0,0065,9906000065
4,-122.357472,47.656118,5,NaN,R1,NaN,NaN,NaN,NaN,NaN,315561,4017,NaN,NaN,1ST,AVE,NW,4017,4017 1ST AVE NW,1ST AVE NW,1STAVENW,40171STAVENW,49.0,8,49.0,8,98107,4908,C010,Z107774670,SEATTLE,SEATTLE,47.656118,-122.357473,1,ESITE:ONETOONE,Single Family,1.264818e+06,243051.203655,KING,990600.0,0060,9906000060


We will only need some of the fields, so let's save what we need before we pull in the rest of the data.

In [3]:
df_zip = df_zip[['ZIP5', 'CTYNAME', 'COUNTY', 'LAT', 'LON']]
df_zip.head()

,ZIP5,CTYNAME,COUNTY,LAT,LON
0,98107,SEATTLE,KING,47.656113,-122.358053
1,98107,SEATTLE,KING,47.656114,-122.357894
2,98107,SEATTLE,KING,47.656114,-122.357731
3,98107,SEATTLE,KING,47.656015,-122.357475
4,98107,SEATTLE,KING,47.656118,-122.357473


Also obtained from the King County GIS Open Data website are King County [Medical Facilities](https://gis-kingcounty.opendata.arcgis.com/datasets/medical-facilities-including-hospitals-medical-facilities-point/data), [Police Stations](https://gis-kingcounty.opendata.arcgis.com/datasets/police-station-locations-in-king-county-kcp-loc-point/data), and [Metro Park-and-Rides](https://gis-kingcounty.opendata.arcgis.com/datasets/king-county-metro-park-and-ride-lots-parkride-point/data). 

**Please Note:** for the medical facilities and park-and-ride data, the city was not listed. In order to obtain the city, I created a field on the csv file itself and typed each address and zip code into google maps. I found the city this way and completed the city field for this data by hand.

Let's pull the data and modify it to only include the fields needed.

#### *Medical Facilities*

In [4]:
df_med = pd.read_csv('Medical_Facilities.csv')
df_med.head()

,Longitude,Latitude,OBJECTID,FEATURE_ID,ESITE,CODE,NAME,ABB_NAME,ADDRESS,ZIPCODE,City,WEBSITE,PIN,MAJOR,MINOR
0,-122.321401,47.612440,1,2655,442834,330,Polyclinic-Broadway,Polyclinic-Broadway,1001 Broadway Ave,98122,Seattle,http://www.polyclinic.com/Locations/SiteInfo/L...,1.978201e+09,197820.0,1270.0
1,-122.229282,47.366847,2,2309,675664,330,Alder Square Health Center,Alder Square Health Center,1404 Central Ave S,98032,Kent,NaN,NaN,NaN,NaN
2,-122.311131,47.607411,3,1076,435003,330,Providence Medical Center,Providence MC,500 17th Ave,98122,Seattle,NaN,NaN,NaN,NaN
3,-122.214881,47.481602,4,2704,582936,330,HealthPoint Renton,HealthPoint Renton,200 S 2nd St,98057,Renton,NaN,NaN,NaN,NaN
4,-122.212243,47.444325,5,2680,608049,330,Valley Medical Clinic Family Medicine,Valley Medical Clinic Family Medicine,3915 Talbot Rd S,98055,Renton,NaN,NaN,NaN,NaN


In [5]:
df_med = df_med[['NAME', 'ZIPCODE', 'City']]
df_med.head()

,NAME,ZIPCODE,City
0,Polyclinic-Broadway,98122,Seattle
1,Alder Square Health Center,98032,Kent
2,Providence Medical Center,98122,Seattle
3,HealthPoint Renton,98057,Renton
4,Valley Medical Clinic Family Medicine,98055,Renton


#### *Police Stations*

In [6]:
df_pol = pd.read_csv('Police_Stations.csv')
df_pol.head()

,Longitude,Latitude,OBJECTID,FEATURE_ID,ESITE,CODE,NAME,ABB_NAME,ADDRESS,ZIPCODE,CITY,STATE_NAME,PHONE,DISTRICT,FAC_CLASS,NEWSTA,RESOLUTION,WEBSITE,PIN,MAJOR,MINOR
0,-122.321530,47.642977,1,650070,419120,65,WSP District 2 Seattle North Detachment,WSP District 2 Seattle N Detatch,811 E Roanoke,98102,Seattle,WA,(206) 720-3040,SE,0,X,placed manually (bldg. believed to be next to ...,http://www.wsp.wa.gov/traveler/d2detach.htm,5535100285,553510,285
1,-122.355348,47.517026,2,650016,500461,65,White Center Storefront,White Center,1327 SW 102nd St,98146,Unincorporated King County,WA,(206) 477-1237,SE,6,X,unsure of location/QC- address matches parcel ...,http://www.kingcounty.gov/depts/sheriff/police...,7973202900,797320,2900
2,-122.304136,47.537379,3,650084,764142,65,KC Airport Law Enforcement,KC Airport Law Enforcement,7277 Perimeter Rd S,98108,Seattle,WA,(206) 296-7392,KC,0,Y,added 9/04,http://www.kingcounty.gov/depts/sheriff/police...,2824049007,282404,9007
3,-122.151958,47.444435,4,650008,611396,65,Fairwood Storefront,Fairwood,17620 140th Ave SE,98058,Unincorporated King County,WA,(206) 477-1243,RN,6,X,QC- address from 17240 140th Ave,http://www.kingcounty.gov/depts/sheriff/police...,7560800010,756080,10
4,-122.044140,47.395092,5,650001,620199,65,KC Precinct 3 Headquarters,KC Precinct 3 HQ,22300 SE 231st St,98038,Maple Valley,WA,(206) 477-6446,MV,1,E,"moved slightly to current location/QC, added n...",http://www.kingcounty.gov/depts/sheriff/police...,1622069143,162206,9143


In [7]:
df_pol = df_pol[['NAME', 'ZIPCODE', 'CITY']]
df_pol.head()

,NAME,ZIPCODE,CITY
0,WSP District 2 Seattle North Detachment,98102,Seattle
1,White Center Storefront,98146,Unincorporated King County
2,KC Airport Law Enforcement,98108,Seattle
3,Fairwood Storefront,98058,Unincorporated King County
4,KC Precinct 3 Headquarters,98038,Maple Valley


#### *Park-and-Rides*

In [8]:
df_par = pd.read_csv('Park_and_Rides.csv')
df_par.head()

,X,Y,OBJECTID,FEATURE_ID,NAME,ADDRESS,ZIP,City,INVENTORY_ID,DISTRICT,OWNERSHIP_TYPE,OWNER,REG_SPACES,ACCESSIBLE,XCOORD,YCOORD,LONGITUDE,LATITUDE,ADDRESS_NUM,ON_PREFIX,ON_STNAME,ON_STYPE,ON_SUFFIX,CROSS_PREFIX,CROSS_STNAME,CROSS_STYPE,CROSS_SUFFIX
0,-122.219876,47.478970,1,570003,Fred Meyer P&R,365 Renton Center Way SW,98057,Renton,533.0,S,L,,21,0.0,1297558.000,177803.0000,-122.219878,47.478968,365.0,,Renter Center,Way,SW,,,,
1,-122.460257,47.451942,2,570013,Ober Park Annex (Vashon Parks Dept) P&R,17130 Vashon Hwy SW,98070,Vashon,737.1,S,L,,9,0.0,1237917.262,169110.1600,-122.460260,47.451940,17106.0,,Vashon,Hwy,SW,,,,
2,-122.305198,47.317920,3,570032,Federal Way Transit Center P&R,31621 23rd Ave S,98003,Federal Way,877.0,S,P,ST,1190,0.0,1275339.000,119460.0000,-122.305201,47.317918,31621.0,,23rd,Ave,S,S,317th,St,
3,-122.257490,47.302090,4,570095,Peasley Canyon Rd/West Valley Hwy P&R,Peasley Canyon Rd/West Valley Hwy,98001,Auburn,739.0,S,P,S,54,0.0,1287064.000,113464.2269,-122.257494,47.302088,0.0,,Peasley Canyon,Rd,,,West Valley,Hwy,
4,-122.233313,47.385157,5,570105,Kent Station Surface Lot P&R,400 1st Ave N,98032,Kent,872.1,S,P,ST,119,0.0,1293613.140,143649.1600,-122.233316,47.385155,301.0,,Railroad,Ave,N,,,,


In [9]:
df_par = df_par[['NAME', 'ZIP', 'City']]
df_par.head()

,NAME,ZIP,City
0,Fred Meyer P&R,98057,Renton
1,Ober Park Annex (Vashon Parks Dept) P&R,98070,Vashon
2,Federal Way Transit Center P&R,98003,Federal Way
3,Peasley Canyon Rd/West Valley Hwy P&R,98001,Auburn
4,Kent Station Surface Lot P&R,98032,Kent


The final piece of data is the restaurant data from Foursquare. In order to pull this data, we need to do some cleaning of the other tables, so we will collect our restaurant data below during preprocessing.

### Data - Preprocessing

Let's go back to our **Zip Code Masterlist**. 

To get it ready for analysis, we will need to accomplish the following:
* Filter for the appropriate locations 
* Make a key field that will define the area
* Remove duplicates
* Sort for ease of use

We will use the cities of Auburn, Burien, Des Moines, Federal Way, Kent, Renton, SeaTac, and Tukwila and the zip codes that coincide with these cities.

For our stakeholders to get a richer understanding, we will not be using city or zip code alone to define our area, but the combination of the two. This allows for more specificity in our analysis, giving the stakeholders a better understanding of where to look. We will call this combined field "City_Zip". 

For the restaurant analysis, Foursquare will use the latitude and longitude of each area. Since we using a created field to define an area, we will need to determine the center of each area to use in analysis.

So, the "center" of each area will be defined according to the latitude and longitude of the zip code table we are using as our masterlist. This zip code table is made up of all the addresses in King County. Once we create the City_Zip field, we can group the data by City_Zip and take the mean of the latitude and longitude. We will do this is a separate table and then add the latitude and longitude back into the main table. **Therefore, we are not defining "center" in terms of geography, but rather in terms of address density.**

Let's complete these steps below.

##### **Filter Data**

In [10]:
# Before we filter, we check the shape so that we will know if our filtering was successful
df_zip.shape

(668781, 5)

In [11]:
# Filter for county
df_zip = df_zip[df_zip.COUNTY.isin(['KING'])]
df_zip.shape

(656206, 5)

In [12]:
# Filter for cities
cities = ['AUBURN', 'BURIEN', 'DES MOINES', 'FEDERAL WAY','KENT', 'RENTON', 'SEATAC', 'TUKWILA']

df_zip = df_zip[df_zip.CTYNAME.isin(cities)]
df_zip.head()

,ZIP5,CTYNAME,COUNTY,LAT,LON
56,98198,DES MOINES,KING,47.398948,-122.318901
57,98198,DES MOINES,KING,47.398769,-122.318915
58,98198,DES MOINES,KING,47.398766,-122.319779
59,98198,DES MOINES,KING,47.398940,-122.319698
60,98198,DES MOINES,KING,47.399161,-122.319915


In [13]:
# Rename columns for ease of use
df_zip.rename(columns={'ZIP5': 'Zip', 
                       'CTYNAME' : 'City', 
                       'COUNTY' : 'County', 
                       'LAT':'Latitude', 
                       'LON': 'Longitude'}, inplace=True)
df_zip.head()

,Zip,City,County,Latitude,Longitude
56,98198,DES MOINES,KING,47.398948,-122.318901
57,98198,DES MOINES,KING,47.398769,-122.318915
58,98198,DES MOINES,KING,47.398766,-122.319779
59,98198,DES MOINES,KING,47.398940,-122.319698
60,98198,DES MOINES,KING,47.399161,-122.319915


##### **Make a Key Field**

In [14]:
# Make a field called "City_Zip" and fill it with the city and zip combo
df_zip['City_Zip'] = df_zip['City'].astype(str) + "-" + df_zip['Zip'].astype(str)
df_zip.head()

,Zip,City,County,Latitude,Longitude,City_Zip
56,98198,DES MOINES,KING,47.398948,-122.318901,DES MOINES-98198
57,98198,DES MOINES,KING,47.398769,-122.318915,DES MOINES-98198
58,98198,DES MOINES,KING,47.398766,-122.319779,DES MOINES-98198
59,98198,DES MOINES,KING,47.398940,-122.319698,DES MOINES-98198
60,98198,DES MOINES,KING,47.399161,-122.319915,DES MOINES-98198


##### **Remove Duplicates**

In [15]:
# Create a separate table for latitude and longitude
df_zip_latlon = df_zip[['Latitude', 'Longitude', 'City_Zip']]
df_zip_latlon.head()

,Latitude,Longitude,City_Zip
56,47.398948,-122.318901,DES MOINES-98198
57,47.398769,-122.318915,DES MOINES-98198
58,47.398766,-122.319779,DES MOINES-98198
59,47.398940,-122.319698,DES MOINES-98198
60,47.399161,-122.319915,DES MOINES-98198


In [16]:
df_zip.drop(columns=['Latitude', 'Longitude'], inplace=True)
df_zip.head()

,Zip,City,County,City_Zip
56,98198,DES MOINES,KING,DES MOINES-98198
57,98198,DES MOINES,KING,DES MOINES-98198
58,98198,DES MOINES,KING,DES MOINES-98198
59,98198,DES MOINES,KING,DES MOINES-98198
60,98198,DES MOINES,KING,DES MOINES-98198


In [17]:
# Find the mean latitude and longitude of each area
df_zip_mean = df_zip_latlon.groupby('City_Zip').mean()
df_zip_mean.head()

,Latitude,Longitude
City_Zip,,
AUBURN-98001,47.322639,-122.257929
AUBURN-98002,47.305466,-122.215589
AUBURN-98030,47.344114,-122.203986
AUBURN-98032,47.349130,-122.226481
AUBURN-98047,47.276322,-122.235774


In [18]:
# Drop duplicates from the main table and add back in longitude and latitude
df_zip.drop_duplicates(subset=['City_Zip'], inplace=True)
df_zip.head()

,Zip,City,County,City_Zip
56,98198,DES MOINES,KING,DES MOINES-98198
68,98166,BURIEN,KING,BURIEN-98166
145,98055,RENTON,KING,RENTON-98055
192,98188,TUKWILA,KING,TUKWILA-98188
198,98002,AUBURN,KING,AUBURN-98002


In [19]:
df_zip= pd.merge(df_zip, df_zip_mean, on='City_Zip', how='left')
df_zip

,Zip,City,County,City_Zip,Latitude,Longitude
0,98198,DES MOINES,KING,DES MOINES-98198,47.388270,-122.315588
1,98166,BURIEN,KING,BURIEN-98166,47.464675,-122.352444
2,98055,RENTON,KING,RENTON-98055,47.446848,-122.200148
3,98188,TUKWILA,KING,TUKWILA-98188,47.457794,-122.263982
4,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589
5,98056,RENTON,KING,RENTON-98056,47.504022,-122.184873
6,98032,KENT,KING,KENT-98032,47.387198,-122.265688
7,98059,RENTON,KING,RENTON-98059,47.492781,-122.151325
8,98023,FEDERAL WAY,KING,FEDERAL WAY-98023,47.308097,-122.361541
9,98030,KENT,KING,KENT-98030,47.369566,-122.191557


##### **Sort**

In [20]:
df_zip.sort_values(by=['City_Zip'], inplace=True)
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude
10,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929
4,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589
36,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986
23,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481
41,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774


Now we will preprocess the Medical Facilities, Police Stations, and Park-and-Rides tables.

For each, we will do the following:
* Rename the columns and filter for pertinent cities
* Make the City-Zip field

For continuity, we will need to make the City uppercase, as the Masterlist table uses an uppercase City in the City_Zip key field.

#### *Medical Facilities*

In [21]:
# Make uppper case and filter for cities
df_med['City'] = df_med['City'].str.upper()
df_med = df_med[df_med.City.isin(cities)]
df_med.head()

,NAME,ZIPCODE,City
1,Alder Square Health Center,98032,KENT
3,HealthPoint Renton,98057,RENTON
4,Valley Medical Clinic Family Medicine,98055,RENTON
5,Schicks Shadel,98146,BURIEN
8,Highline Medical Center Speciality Campus,98168,TUKWILA


In [22]:
# Make City_Zip field
df_med.rename(columns={'NAME':'Facility','ZIPCODE':'Zip'}, inplace=True)
df_med['City_Zip'] = df_med['City'].astype(str) + "-" + df_med['Zip'].astype(str)
df_med.head()

,Facility,Zip,City,City_Zip
1,Alder Square Health Center,98032,KENT,KENT-98032
3,HealthPoint Renton,98057,RENTON,RENTON-98057
4,Valley Medical Clinic Family Medicine,98055,RENTON,RENTON-98055
5,Schicks Shadel,98146,BURIEN,BURIEN-98146
8,Highline Medical Center Speciality Campus,98168,TUKWILA,TUKWILA-98168


#### *Police Stations*

In [23]:
# Make uppper case and filter for cities
df_pol.rename(columns={'NAME':'Facility','ZIPCODE':'Zip', 'CITY':'City'}, inplace=True)

df_pol['City'] = df_pol['City'].str.upper()
df_pol = df_pol[df_pol.City.isin(cities)]
df_pol.head()

,Facility,Zip,City
11,SeaTac Police Department,98188,SEATAC
12,WSP District 2 Seattle South Detachment,98188,TUKWILA
18,Corrections Facility (Jail),98030,KENT
25,Auburn Police Department,98002,AUBURN
29,Renton Police Department,98057,RENTON


In [24]:
# Make City_Zip field
df_pol['City_Zip'] = df_pol['City'].astype(str) + "-" + df_pol['Zip'].astype(str)
df_pol.head()

,Facility,Zip,City,City_Zip
11,SeaTac Police Department,98188,SEATAC,SEATAC-98188
12,WSP District 2 Seattle South Detachment,98188,TUKWILA,TUKWILA-98188
18,Corrections Facility (Jail),98030,KENT,KENT-98030
25,Auburn Police Department,98002,AUBURN,AUBURN-98002
29,Renton Police Department,98057,RENTON,RENTON-98057


#### *Park-and-Rides*

In [25]:
# Make uppper case and filter for cities
df_par.rename(columns={'NAME':'Facility', 'ZIP': 'Zip'}, inplace=True)

df_par['City'] = df_par['City'].str.upper()
df_par = df_par[df_par.City.isin(cities)]
df_par.head()

,Facility,Zip,City
0,Fred Meyer P&R,98057,RENTON
2,Federal Way Transit Center P&R,98003,FEDERAL WAY
3,Peasley Canyon Rd/West Valley Hwy P&R,98001,AUBURN
4,Kent Station Surface Lot P&R,98032,KENT
8,Burien TC P&R,98166,BURIEN


In [26]:
# Make City_Zip field
df_par['City_Zip'] = df_par['City'].astype(str) + '-' + df_par['Zip'].astype(str)
df_par.head()

,Facility,Zip,City,City_Zip
0,Fred Meyer P&R,98057,RENTON,RENTON-98057
2,Federal Way Transit Center P&R,98003,FEDERAL WAY,FEDERAL WAY-98003
3,Peasley Canyon Rd/West Valley Hwy P&R,98001,AUBURN,AUBURN-98001
4,Kent Station Surface Lot P&R,98032,KENT,KENT-98032
8,Burien TC P&R,98166,BURIEN,BURIEN-98166


#### *Restaurants*

Now that we have the initial data pre-processed, we will pull the restaurant data from Foursquare and preprocess it.

A few things to note about the restaurant data:
* As stated above, the Foursqure API will use latitude and longitude coordinates to pull restaurants within a certain radius of the center of each area. 
* There is a limit of 100 venues that can be pulled for each area provided. Bear in mind that even if the data says there are 100 restaurants in an city-zip area, there may be more restaurants in that area that are not pulled because of the limit.
* In order to get the best data while considering the limit applied but to still measure realistically, we have chosen to use 2 miles, or about 3219 meters as our radius.

In [27]:
# Store Foursquare credentials (hidden to protect credentials)

In [28]:
# Store credentials

Your credentials are stored


In [29]:
# Define method for pulling restaurant data for each city-zip
def getNearbyVenues(names, latitudes, longitudes, radius=3219):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=restaurant'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City_Zip', 
                  'City_Zip Latitude', 
                  'City_Zip Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
# Use the method to get the restaurant data
df_res = getNearbyVenues(names=df_zip['City_Zip'], latitudes=df_zip['Latitude'], longitudes=df_zip['Longitude'])
df_res.head()

,City_Zip,City_Zip Latitude,City_Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AUBURN-98001,47.322639,-122.257929,Sun Break Cafe,47.306689,-122.230740,Café
1,AUBURN-98001,47.322639,-122.257929,PICK-QUICK Drive In,47.318346,-122.221001,Burger Joint
2,AUBURN-98001,47.322639,-122.257929,Sushido,47.307719,-122.228011,Sushi Restaurant
3,AUBURN-98001,47.322639,-122.257929,Qdoba Mexican EATS,47.296154,-122.242884,Mexican Restaurant
4,AUBURN-98001,47.322639,-122.257929,Herfy's Hamburgers and Seafood,47.305435,-122.228872,American Restaurant


## Methodology <a name='methodology'></a>

Having prepared the data for analysis, we will look at the general structure and areas of what we have found, observing how the city-zip code delineation fits with the overall South King County area. We will also **view a map of the area** to provide a visual reference.

Each amenity **will be scored** based on how many of each exist in each city-zip code compared to how many total are in the South King County area. Those scores will totaled to give each city-zip a total score. Each score will be divided into tiers and mapped. Those tiers will be analyzed for any patterns and their implications discussed.

## Analysis <a name='analysis'></a>

In general, we will start by **presenting a map** and some general analysis of the data we are using. Then, the data will be scored and tiered, with the results displayed in tables and maps in the Results section.

#### *Map*

In [31]:
# Find latitude and longitude of somewhere near the center of our cities, "Kent, WA" 
# Uses geolocator (hidden to protect credentials)

In [32]:
# Geocoder credentials

Geograpical coordinates near the center of the South King County cities are 47.3826903, -122.2270272.


In [33]:
# Make a map of our area with markers to show the city-zip codes
map_kingcounty = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, city_zip in zip(df_zip['Latitude'], df_zip['Longitude'], df_zip['City_Zip']):
    label = '{}'.format(city_zip)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kingcounty)  
    
map_kingcounty

After reviewing the map, we can see that there are 2 outlier city-zip codes: Kent-98051 and Kent-98038. Our goal is to find areas that are good for Seattle commuters. Since these outliers are away from the main thoroughfares that lead to Seattle and would result in far too long of a commute time, we will exclude them from our analysis, based on these observations.

In [34]:
df_zip = df_zip[~df_zip.City_Zip.isin(['KENT-98051', 'KENT-98038'])]
df_med = df_med[~df_med.City_Zip.isin(['KENT-98051', 'KENT-98038'])]
df_pol = df_pol[~df_pol.City_Zip.isin(['KENT-98051', 'KENT-98038'])]
df_par = df_par[~df_par.City_Zip.isin(['KENT-98051', 'KENT-98038'])]
df_res = df_res[~df_res.City_Zip.isin(['KENT-98051', 'KENT-98038'])]

We can now do some preliminary observational analysis on the city-zip field.

##### **How many city-zip codes per city?**

In [35]:
df_cityzip_by_city = df_zip[['City_Zip', 'City']]
df_cityzip_by_city = df_cityzip_by_city.groupby('City').count()
df_cityzip_by_city

,City_Zip
City,
AUBURN,6
BURIEN,4
DES MOINES,4
FEDERAL WAY,6
KENT,7
RENTON,7
SEATAC,5
TUKWILA,7


##### **How many cities in each zip code?**

In [36]:
# First, break the cities up using one-hot encoding
df_city_onehot = pd.get_dummies(df_zip[['City']].astype(str), prefix="", prefix_sep="")

c_col_name = 'Zip'
c_first_col = df_zip['Zip']
df_city_onehot.insert(0, c_col_name, c_first_col)

df_city_onehot['Zip'] = df_zip['Zip']

df_city_onehot.head()

,Zip,AUBURN,BURIEN,DES MOINES,FEDERAL WAY,KENT,RENTON,SEATAC,TUKWILA
10,98001,1,0,0,0,0,0,0,0
4,98002,1,0,0,0,0,0,0,0
36,98030,1,0,0,0,0,0,0,0
23,98032,1,0,0,0,0,0,0,0
41,98047,1,0,0,0,0,0,0,0


In [37]:
# Then find the total of each city in each zip code
df_city_totals = df_city_onehot.groupby('Zip').sum()

df_city_totals['Total'] = (df_city_totals['AUBURN']+
                           df_city_totals['BURIEN']+
                           df_city_totals['DES MOINES']+
                           df_city_totals['FEDERAL WAY']+
                           df_city_totals['KENT']+
                           df_city_totals['RENTON']+
                           df_city_totals['SEATAC']+
                           df_city_totals['TUKWILA'])
df_city_totals

,AUBURN,BURIEN,DES MOINES,FEDERAL WAY,KENT,RENTON,SEATAC,TUKWILA,Total
Zip,,,,,,,,,
98001,1,0,0,1,0,0,0,0,2
98002,1,0,0,0,0,0,0,0,1
98003,0,0,1,1,0,0,0,0,2
98023,0,0,0,1,0,0,0,0,1
98030,1,0,0,0,1,0,0,0,2
98031,0,0,0,0,1,1,0,0,2
98032,1,0,1,1,1,0,0,1,5
98042,0,0,0,0,1,0,0,0,1
98047,1,0,0,0,0,0,0,0,1


To see how just how many cities total comprise each zip code:

In [38]:
df_city_by_zip = df_city_totals.reset_index()
df_city_by_zip = df_city_by_zip[["Zip", "Total"]]
df_city_by_zip

,Zip,Total
0,98001,2
1,98002,1
2,98003,2
3,98023,1
4,98030,2
5,98031,2
6,98032,5
7,98042,1
8,98047,1
9,98055,2


##### **Some Observations**

* Now that we have filtered the outliers, all of the city-zip code areas are within a general area of each other
* All the cities have 4 - 7 zip codes within them
* All the zip codes have 1 - 3 cities within them

Things seem decently spread out and even for our purposes. There don't seem to be any more stand-out outliers nor any data anomalies that jump out. We can now begin the scoring and tiering.

### Scoring

To score the Medical Facilities (Med_Score), Police Stations (Pol_Score), and Park-and-Rides (Par_Score), we will need to complete the following steps:
* Get a count of how many amenities are in each city-zip code
* Score them by dividing the number of amenties in each area by the total amenities in all the areas, times 100
* Create a separate score table and merge it with the master zip code table
* Replace NaN scores with 0

Note, we have chosen to score this way so as to normalize the scoring so that no one amenity as a greater weight than any other. We also have chosen to score a 0 to any area that does not have that amenity present at all. Adding these scores back to one table allows for easy score analysis and tiering.

#### *Medical Facilities*

In [39]:
# Get a count
df_med_ct = df_med.groupby('City_Zip').count()
df_med_ct.sort_values(['Facility'], ascending=False, axis=0, inplace=True)
df_med_ct.head()

,Facility,Zip,City
City_Zip,,,
FEDERAL WAY-98003,7,7,7
AUBURN-98002,4,4,4
RENTON-98055,4,4,4
KENT-98030,3,3,3
KENT-98032,3,3,3


In [40]:
# Calculate the score
med_sum = sum(df_med_ct['Facility'])
df_med_ct['Med_Score'] = (df_med_ct['Facility']/med_sum)*100
df_med_ct

,Facility,Zip,City,Med_Score
City_Zip,,,,
FEDERAL WAY-98003,7,7,7,21.212121
AUBURN-98002,4,4,4,12.121212
RENTON-98055,4,4,4,12.121212
KENT-98030,3,3,3,9.090909
KENT-98032,3,3,3,9.090909
BURIEN-98166,2,2,2,6.060606
SEATAC-98188,2,2,2,6.060606
BURIEN-98146,1,1,1,3.030303
DES MOINES-98198,1,1,1,3.030303


In [41]:
# Merge with master list
df_med_score = df_med_ct.reset_index()
df_med_score = df_med_score[['City_Zip', 'Med_Score']]

df_zip = pd.merge(df_zip, df_med_score, on ='City_Zip', how ='left') 
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,NaN
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,NaN
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,NaN
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,NaN


In [42]:
# Replace NaN
df_zip['Med_Score'] = df_zip['Med_Score'].replace(np.nan, 0)
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000


#### *Police Stations*

In [43]:
# Get a count
df_pol_ct = df_pol.groupby('City_Zip').count()
df_pol_ct.sort_values(['Facility'], ascending=False, axis=0, inplace=True)
df_pol_ct.head()

,Facility,Zip,City
City_Zip,,,
KENT-98030,2,2,2
KENT-98032,2,2,2
TUKWILA-98188,2,2,2
AUBURN-98002,1,1,1
BURIEN-98166,1,1,1


In [44]:
# Calculate the score
df_pol_ct['Pol_Score'] = df_pol_ct['Facility']/sum(df_pol_ct['Facility'])*100
df_pol_ct

,Facility,Zip,City,Pol_Score
City_Zip,,,,
KENT-98030,2,2,2,13.333333
KENT-98032,2,2,2,13.333333
TUKWILA-98188,2,2,2,13.333333
AUBURN-98002,1,1,1,6.666667
BURIEN-98166,1,1,1,6.666667
BURIEN-98168,1,1,1,6.666667
DES MOINES-98198,1,1,1,6.666667
FEDERAL WAY-98003,1,1,1,6.666667
RENTON-98057,1,1,1,6.666667


In [45]:
# Merge with master list
df_pol_score = df_pol_ct.reset_index()
df_pol_score = df_pol_score[['City_Zip', 'Pol_Score']]

df_zip = pd.merge(df_zip, df_pol_score, on='City_Zip', how='left')
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score,Pol_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000,NaN
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000,NaN
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000,NaN
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000,NaN


In [46]:
# Replace NaN
df_zip['Pol_Score'] = df_zip['Pol_Score'].replace(np.nan, 0)
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score,Pol_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000,0.000000
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000,0.000000
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000,0.000000
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000,0.000000


#### *Park-and-Rides*

In [47]:
# Get a count
df_par_ct = df_par.groupby('City_Zip').count()
df_par_ct.sort_values(['Facility'], ascending=False, axis=0, inplace=True)
df_par_ct.head()

,Facility,Zip,City
City_Zip,,,
FEDERAL WAY-98003,7,7,7
KENT-98032,6,6,6
AUBURN-98001,4,4,4
KENT-98031,4,4,4
AUBURN-98092,3,3,3


In [48]:
# Calculate the score
df_par_ct['Par_Score'] = df_par_ct['Facility']/sum(df_par_ct['Facility'])*100
df_par_ct

,Facility,Zip,City,Par_Score
City_Zip,,,,
FEDERAL WAY-98003,7,7,7,15.217391
KENT-98032,6,6,6,13.043478
AUBURN-98001,4,4,4,8.695652
KENT-98031,4,4,4,8.695652
AUBURN-98092,3,3,3,6.521739
RENTON-98055,3,3,3,6.521739
RENTON-98057,2,2,2,4.347826
TUKWILA-98168,2,2,2,4.347826
SEATAC-98188,2,2,2,4.347826


In [49]:
# Merge with master list
df_par_score = df_par_ct.reset_index()
df_par_score = df_par_score[['City_Zip', 'Par_Score']]

df_zip = pd.merge(df_zip, df_par_score, on='City_Zip', how='left')
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000,0.000000,8.695652
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000,0.000000,NaN
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000,0.000000,NaN
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000,0.000000,NaN


In [50]:
# Replace NaN
df_zip['Par_Score'] = df_zip['Par_Score'].replace(np.nan, 0)
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000,0.000000,8.695652
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000,0.000000,0.000000
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000,0.000000,0.000000
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000,0.000000,0.000000


#### *Restaurants*

For restaurants, we will use a similar technique, except in the scoring. Since the restaurant data is based on proximity to the center of an area, restaurants could be counted more than once for each area. Therefore, we will need to find the number of unique restaurants first, then score using that number as the divisor. 

**Please note:** as referenced above, there is a 100 restaurant limit. Therefore, every area that has 100 restaurants reported must be mentally treated as 100 or more. This will, somewhat, diminish the accuracy of the scoring, but still provides a decent picture for realtors and a good springboard for home buyers.

In [51]:
# Get a count
df_res_ct = df_res.groupby('City_Zip').count()
df_res_ct.sort_values(['Venue'], ascending=False, axis=0, inplace=True)
df_res_ct.head()

,City_Zip Latitude,City_Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City_Zip,,,,,,
TUKWILA-98188,100,100,100,100,100,100
BURIEN-98148,100,100,100,100,100,100
AUBURN-98002,100,100,100,100,100,100
RENTON-98178,100,100,100,100,100,100
SEATAC-98158,100,100,100,100,100,100


In [52]:
# Calculate the score
n = len(pd.unique(df_res['Venue'])) 
  
print("No.of.unique venues :", n)

No.of.unique venues : 908


In [53]:
df_res_ct['Res_Score'] = df_res_ct['Venue']/n*100
df_res_ct

,City_Zip Latitude,City_Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Res_Score
City_Zip,,,,,,,
TUKWILA-98188,100,100,100,100,100,100,11.013216
BURIEN-98148,100,100,100,100,100,100,11.013216
AUBURN-98002,100,100,100,100,100,100,11.013216
RENTON-98178,100,100,100,100,100,100,11.013216
SEATAC-98158,100,100,100,100,100,100,11.013216
FEDERAL WAY-98003,100,100,100,100,100,100,11.013216
SEATAC-98188,100,100,100,100,100,100,11.013216
FEDERAL WAY-98001,100,100,100,100,100,100,11.013216
RENTON-98057,100,100,100,100,100,100,11.013216


In [54]:
# Merge with master list
df_res_score = df_res_ct.reset_index()
df_res_score = df_res_score[['City_Zip', 'Res_Score']]

df_zip = pd.merge(df_zip, df_res_score, on ='City_Zip', how ='left')
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score,Res_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000,0.000000,8.695652,6.277533
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913,11.013216
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000,0.000000,0.000000,6.938326
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000,0.000000,0.000000,4.735683
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000,0.000000,0.000000,7.268722


In [55]:
# Replace NaN
df_zip['Res_Score'] = df_zip['Res_Score'].replace(np.nan, 0)
df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score,Res_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000,0.000000,8.695652,6.277533
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913,11.013216
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000,0.000000,0.000000,6.938326
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000,0.000000,0.000000,4.735683
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000,0.000000,0.000000,7.268722


#### *Final Score*

In [56]:
# Create a total column
df_zip['Total_Score'] = (df_zip['Med_Score'].astype(float)+
                         df_zip['Pol_Score'].astype(float)+
                         df_zip['Par_Score'].astype(float)+
                         df_zip['Res_Score'].astype(float))

df_zip.head()

,Zip,City,County,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score,Res_Score,Total_Score
0,98001,AUBURN,KING,AUBURN-98001,47.322639,-122.257929,0.000000,0.000000,8.695652,6.277533,14.973185
1,98002,AUBURN,KING,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913,11.013216,31.975008
2,98030,AUBURN,KING,AUBURN-98030,47.344114,-122.203986,0.000000,0.000000,0.000000,6.938326,6.938326
3,98032,AUBURN,KING,AUBURN-98032,47.349130,-122.226481,0.000000,0.000000,0.000000,4.735683,4.735683
4,98047,AUBURN,KING,AUBURN-98047,47.276322,-122.235774,0.000000,0.000000,0.000000,7.268722,7.268722


In [57]:
# Make a winner's table to determine winner
df_winner = df_zip[['City_Zip','Latitude', 'Longitude', 'Med_Score', 'Pol_Score', 'Par_Score', 'Res_Score', 'Total_Score']]
df_winner.sort_values(['Total_Score'], ascending=False, axis=0, inplace=True)
df_winner

<ipython-input-57-59dbaa61973a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_winner.sort_values(['Total_Score'], ascending=False, axis=0, inplace=True)


,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score,Res_Score,Total_Score
15,FEDERAL WAY-98003,47.323493,-122.314357,21.212121,6.666667,15.217391,11.013216,54.109395
22,KENT-98032,47.387198,-122.265688,9.090909,13.333333,13.043478,8.149780,43.617500
1,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913,11.013216,31.975008
45,TUKWILA-98188,47.457794,-122.263982,3.030303,13.333333,4.347826,11.013216,31.724678
20,KENT-98030,47.369566,-122.191557,9.090909,13.333333,0.000000,6.497797,28.922040
37,SEATAC-98188,47.444513,-122.284091,6.060606,6.666667,4.347826,11.013216,28.088315
30,RENTON-98057,47.479239,-122.212348,3.030303,6.666667,4.347826,11.013216,25.058012
28,RENTON-98055,47.446848,-122.200148,12.121212,0.000000,6.521739,5.616740,24.259691
8,BURIEN-98166,47.464675,-122.352444,6.060606,6.666667,2.173913,8.920705,23.821891
43,TUKWILA-98168,47.480437,-122.279235,3.030303,6.666667,4.347826,5.726872,19.771668


In [58]:
# Make a top 5 table
df_top_5 = df_winner.head()
df_top_5

,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score,Res_Score,Total_Score
15,FEDERAL WAY-98003,47.323493,-122.314357,21.212121,6.666667,15.217391,11.013216,54.109395
22,KENT-98032,47.387198,-122.265688,9.090909,13.333333,13.043478,8.149780,43.617500
1,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913,11.013216,31.975008
45,TUKWILA-98188,47.457794,-122.263982,3.030303,13.333333,4.347826,11.013216,31.724678
20,KENT-98030,47.369566,-122.191557,9.090909,13.333333,0.000000,6.497797,28.922040


#### *Tiers*

As another way of viewing the data, we are going to divide the city-zip codes into tiers, based on score:
* Tier 1: > 25 score
* Tier 2: > 15 score
* Tier 3: > 10 score
* Tier 4: > 5 score
* Tier 5: 5 or below score

Once divided, we will further analyze based on tier.

In [59]:
# Add Tiers to winners table
win_conditions = [
    (df_winner['Total_Score'] > 25),
    (df_winner['Total_Score'] <= 25) & (df_winner['Total_Score'] > 15),
    (df_winner['Total_Score'] <= 15) & (df_winner['Total_Score'] > 10),
    (df_winner['Total_Score'] <= 10) & (df_winner['Total_Score'] > 5),
    (df_winner['Total_Score'] <=5)
    ]

win_values = [1, 2, 3, 4, 5]

df_add_tier = df_winner
df_add_tier['Tier'] = np.select(win_conditions, win_values)

df_add_tier.head()

<ipython-input-59-72b64cfc9f51>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_add_tier['Tier'] = np.select(win_conditions, win_values)


,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score,Res_Score,Total_Score,Tier
15,FEDERAL WAY-98003,47.323493,-122.314357,21.212121,6.666667,15.217391,11.013216,54.109395,1
22,KENT-98032,47.387198,-122.265688,9.090909,13.333333,13.043478,8.149780,43.617500,1
1,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913,11.013216,31.975008,1
45,TUKWILA-98188,47.457794,-122.263982,3.030303,13.333333,4.347826,11.013216,31.724678,1
20,KENT-98030,47.369566,-122.191557,9.090909,13.333333,0.000000,6.497797,28.922040,1


#### *Tier Analysis*

In [60]:
# Make a tier analysis table
df_tier = df_add_tier[['City_Zip', 'Tier']]

df_tier.head()

,City_Zip,Tier
15,FEDERAL WAY-98003,1
22,KENT-98032,1
1,AUBURN-98002,1
45,TUKWILA-98188,1
20,KENT-98030,1


In [61]:
df_tier_merger = df_zip[['City_Zip', 'City', 'Zip']]
df_tier_merger.head()

,City_Zip,City,Zip
0,AUBURN-98001,AUBURN,98001
1,AUBURN-98002,AUBURN,98002
2,AUBURN-98030,AUBURN,98030
3,AUBURN-98032,AUBURN,98032
4,AUBURN-98047,AUBURN,98047


In [62]:
df_tier_an = pd.merge(df_tier_merger, df_tier, on='City_Zip', how='left')
df_tier_an.head()

,City_Zip,City,Zip,Tier
0,AUBURN-98001,AUBURN,98001,3
1,AUBURN-98002,AUBURN,98002,1
2,AUBURN-98030,AUBURN,98030,4
3,AUBURN-98032,AUBURN,98032,5
4,AUBURN-98047,AUBURN,98047,4


Let's also use onehot encoding to make a table that will let us analyze what location is in what tier.

In [63]:
df_tier_onehot = pd.get_dummies(df_tier_an[['Tier']].astype(str), prefix="", prefix_sep="")

t_col_name = 'City'
t_first_col = df_tier_an['City']
df_tier_onehot.insert(0, t_col_name, t_first_col)

df_tier_onehot['City'] = df_tier_an['City']

t_col2_name = 'Zip'
t_second_col = df_tier_an['Zip']
df_tier_onehot.insert(1, t_col2_name, t_second_col)

df_tier_onehot['Zip'] = df_tier_an['Zip']

df_tier_onehot.head()

,City,Zip,1,2,3,4,5
0,AUBURN,98001,0,0,1,0,0
1,AUBURN,98002,1,0,0,0,0
2,AUBURN,98030,0,0,0,1,0
3,AUBURN,98032,0,0,0,0,1
4,AUBURN,98047,0,0,0,1,0


In [64]:
df_tier_onehot.rename(columns={'1':'Tier 1', '2':'Tier 2', '3':'Tier 3', '4':'Tier 4','5':'Tier 5'}, inplace=True)
df_tier_onehot.head()

,City,Zip,Tier 1,Tier 2,Tier 3,Tier 4,Tier 5
0,AUBURN,98001,0,0,1,0,0
1,AUBURN,98002,1,0,0,0,0
2,AUBURN,98030,0,0,0,1,0
3,AUBURN,98032,0,0,0,0,1
4,AUBURN,98047,0,0,0,1,0


Now that we have the tables needed prepared, let's look at the data and make observations.

##### **How many city-zip areas are in each tier?**

In [65]:
df_cityzip_by_tier = df_tier_an[['City_Zip', 'Tier']]
df_cityzip_by_tier = df_cityzip_by_tier.groupby('Tier').count()
df_cityzip_by_tier

,City_Zip
Tier,
1,7
2,7
3,11
4,14
5,7


##### **How many of each tier are in each city?**

In [66]:
df_tier_by_city = df_tier_onehot.drop(columns = 'Zip')
df_tier_by_city = df_tier_by_city.groupby('City').sum()
df_tier_by_city['Total'] = (df_tier_by_city['Tier 1'] + 
                            df_tier_by_city['Tier 2'] + 
                            df_tier_by_city['Tier 3'] + 
                            df_tier_by_city['Tier 4'] + 
                            df_tier_by_city['Tier 5'])
df_tier_by_city

,Tier 1,Tier 2,Tier 3,Tier 4,Tier 5,Total
City,,,,,,
AUBURN,1,0,2,2,1,6
BURIEN,0,1,3,0,0,4
DES MOINES,0,1,0,2,1,4
FEDERAL WAY,1,0,1,2,2,6
KENT,2,1,1,2,1,7
RENTON,1,2,3,1,0,7
SEATAC,1,1,0,3,0,5
TUKWILA,1,1,1,2,2,7


##### **How many of each tier are in each zip code?**

In [67]:
df_tier_by_zip = df_tier_onehot.drop(columns = 'City')
df_tier_by_zip = df_tier_by_zip.groupby('Zip').sum()
df_tier_by_zip['Total'] = (df_tier_by_zip['Tier 1'] + 
                           df_tier_by_zip['Tier 2'] + 
                           df_tier_by_zip['Tier 3'] + 
                           df_tier_by_zip['Tier 4'] + 
                           df_tier_by_zip['Tier 5'])
df_tier_by_zip

,Tier 1,Tier 2,Tier 3,Tier 4,Tier 5,Total
Zip,,,,,,
98001,0,0,2,0,0,2
98002,1,0,0,0,0,1
98003,1,0,0,0,1,2
98023,0,0,0,1,0,1
98030,1,0,0,1,0,2
98031,0,1,0,1,0,2
98032,1,0,0,2,2,5
98042,0,0,0,1,0,1
98047,0,0,0,1,0,1


## Results <a name="results"></a>

The data has been reviewed, scored, and tiered. To give us a sense of the results, lets look at a map of what this tiered data looks like.

In [68]:
map_tiers = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(1,6)]
colors_array = cm.gist_rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, cz, tier in zip(df_winner['Latitude'], df_winner['Longitude'], df_winner['City_Zip'], df_winner['Tier']):
    label = folium.Popup(str(cz) + ' - Tier ' + str(tier), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[tier-1],
        fill=True,
        fill_color=rainbow[tier-1],
        fill_opacity=0.7).add_to(map_tiers)
       
map_tiers

##### _LEGEND_
##### Tier 1: Red, Score: > 25 | Tier 2: Yellow, Score: 25 - 15 | Tier 3: Green, Score: 15 - 10 | Tier 4: Blue, Score: 10 - 5 | Tier 5: Pink, Score: < 5


The area with the **highest total score** is Federal Way-98003. The area with the **lowest total score** is Federal Way-98422.
Here is a list of the **top 5** best areas in South King County:

In [69]:
df_top_5

,City_Zip,Latitude,Longitude,Med_Score,Pol_Score,Par_Score,Res_Score,Total_Score
15,FEDERAL WAY-98003,47.323493,-122.314357,21.212121,6.666667,15.217391,11.013216,54.109395
22,KENT-98032,47.387198,-122.265688,9.090909,13.333333,13.043478,8.149780,43.617500
1,AUBURN-98002,47.305466,-122.215589,12.121212,6.666667,2.173913,11.013216,31.975008
45,TUKWILA-98188,47.457794,-122.263982,3.030303,13.333333,4.347826,11.013216,31.724678
20,KENT-98030,47.369566,-122.191557,9.090909,13.333333,0.000000,6.497797,28.922040


The top area for each score is:
* Highest Score for Medical Facilities: Federal Way-98003
* Highest Score for Police Stations: Kent-98032, Tukwila-98188, Kent-98030
* Highest Score for Park-and-Rides: Federal Way-98003
* Highest Score for Restaurants: 10 different locations reached the 100 limit, spanning from Tier 1 - 3

Here are some other observations and results from our analysis:
* The city with the highest score is also the city with the lowest score.
* Within the top tier, Kent-98030 has at least 1 amenity with a score of 0 (park-and-ride)
* Besides those in the top tier, 2 other areas have at least 1 of each amenity: Tukwila-98168 and Burien-98166
* The city of Renton has only 1 area out of 7 in the top tier (the last place of the top tier), but no areas in Tier 5
* Both the areas in the 98188 zip code are in Tier 1 (Tukwila-98188 and SeaTac-98188)
* 2 cities have no Tier 1 areas: Burien and Des Moines

## Discussion <a name="discussion"></a>

The analysis of this data shows that Federal Way-98003 has the highest amenity score and therefore the best chance of meeting the needs of a Seattle commuter.

Interestingly, our research has also pointed out the necessity when reviewing an area for a potential home of looking past just the city name or zip code. We have many examples in our data of cities and zip codes spanning across the tiers, suggesting that the greater specifity gained from using city and zip code has given us a more accurate picture of what each area has to offer. For example, the city of Kent shows scores as high as 43.6 and as low as 3.5, with areas in each of the 5 tiers. If a homebuyer were to ask a realtor what sort of amenties are available in the city of Kent, the answer would truly vary from area to area of Kent. Similar wide spreads are seen in other cities and when considering zip codes. In fact, the city with highest score was also the city with the lowest score! Clearly, **you can't judge an area by its city or zip.**

This trend of a wide spread is not 100% across the board. We found that for zip code 98188 that spans parts of Tukwila and SeaTac, that both areas considered were in Tier 1. We also found that both the cities of Burien and Des Moines held no areas that reached up to Tier 1 status, indicating that there are fewer amenities available in their areas, or at least that amenities may be spread out in their areas instead of concentrated, as found in a big city downtown scenario. 

**More research could be done** to delve deeper into and provide even more specificity to the analysis. For example: restaurant scores were given for restaurants in a 2 mile radius of the center of the city-zip code areas, and there was a cap at 100 restaurants. 10 of the locations reached that cap. Without a cap, restaurants at 5 or 10 miles could be evaluated and scored with more accuracy. 

Further, the location from the nearest amenity could be added as a partial score if a particular area doesn't have one that amenity within its own borders. In lieu of looking for amenities that reside within that city-zip code, locations that are within a certain number of miles from that area could be considered for all the amenities, not just restaurants. 

Other amenities could also be examined, depending upon the specific needs of a narrower audience base, such as art galleries for the art enthusiasts, or gas stations for those who like to drive into work instead of using public transit. There are many more variations and scope this type of examination could lend itself to.

## Conclusion <a name="conclusion"></a>

The city of Seattle is an attractive place for homebuyers. Our goal was to provide aid for a less-expensive alternative to living in the big city for those who work downtown. We set out to provide both homebuyers and realtors insight into the areas of South King County that would provide some of the amenities of downtown, including commuting options, without the price tag.

By dividing each area by both its city and zip code, we were able to give specific analysis of the locations in this vicinity. We scored each and came up with a working list by tier. 

We believe this information is ready to be used by potential homebuyers to give them an idea of where to begin their search. Realtors also can use this information to provide selling points and guide their city-worker buyers to these areas. 

This study is only the tip of an iceberg of potential to understanding what is available and fine-tuning location hunting for homebuyers everywhere. We hope the future will find many more reports like these coming to the aid of those who wish to enjoy a lifestyle at any price point. We are happy and proud to have begun this highly useful and widely applicable work.